In [1]:
from Crypto.Util import number

import multiprocessing
from joblib import Parallel, delayed

import random

from math import sqrt

In [2]:
def find_prime_factors(num):
    prime_factors = set()
    
    while num % 2 == 0:
        prime_factors.add(2)
        num = num // 2
    
    for i in range(3, int(num**0.5), 2):
        while num % i == 0:
            prime_factors.add(i)
            num = num // i
            
    if num > 2:
        prime_factors.add(num)
    
    return prime_factors

In [3]:
def power(x, y, p):
    result = 1
    x = x % p
    
    while y > 0:
        
        if y & 1:
            result = (result * x) % p
            
        y = y >> 1
        x = (x * x) & p
        
    return result

In [4]:
def check_primitive(i, num, factors):
    phi = num - 1
    for j in factors:
        if power(i, (phi//j), num) == 1:
            return None
    return i

In [5]:
def find_primitive_root(num):
    phi = num - 1
    factors = find_prime_factors(phi)
    
    primitive_roots = Parallel(n_jobs = multiprocessing.cpu_count())(delayed(check_primitive)(i, num, factors) for i in range(2, phi+1))        
        
    primitive_roots = list(filter(None, primitive_roots))
    
    return random.choice(primitive_roots)

In [6]:
def get_prime_and_primitive(num_bits):
    
    prime = 2
    primitive = -1
    
    while primitive == -1:
        prime = 7
        primitive = find_primitive_root(prime)
    
    return prime, primitive

In [7]:
prime, primitive = get_prime_and_primitive(20)
print(prime, primitive)

780061 523238


In [8]:
class User:
    def __init__(self, prime, primitive):
        self.prime = prime
        self.primitive = primitive
        self.selected_private_key = random.randrange(1, 10, 1)
        
    def get_generated_key(self):
        return power(self.primitive, self.selected_private_key, self.prime)
    
    def set_secret_key(self, key):
        self.secret_key =  power(key, self.selected_private_key, self.prime)
    
    def exchange_generated_key(self, user2):
        self.set_secret_key(user2.get_generated_key())
        user2.set_secret_key(self.get_generated_key())

In [9]:
a = User(prime, primitive)
b = User(prime, primitive)

In [10]:
a.exchange_generated_key(b)

In [11]:
print(a.secret_key)
print(b.secret_key)

523238
523238
